#### Imports

In [1]:
import pandas as pd
import requests
import json

#### Function to get box score data

In [2]:
def getStdBoxScores(season, team):
    url = "https://stats.nba.com/stats/teamgamelogs"
    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Origin": "https://www.nba.com",
        "Referer": "https://www.nba.com/",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.3.1 Safari/605.1.15"
    }

    params = {
        "DateFrom": "",
        "DateTo": "",
        "Season": season,
        "SeasonType": "Regular Season",
        "TeamID": team
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data['resultSets'][0]['rowSet'], columns=data['resultSets'][0]['headers'])
    else:
        print(f"Request failed with status code {response.status_code}")



In [3]:
def getAdvBoxScores(season, team):  
    url = "https://stats.nba.com/stats/teamgamelogs"
    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Origin": "https://www.nba.com",
        "Referer": "https://www.nba.com/",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.3.1 Safari/605.1.15"
    }

    params = {
        "DateFrom": "",
        "DateTo": "",
        "GameSegment": "",
        "ISTRound": "",
        "LastNGames": "0",
        "LeagueID": "00",
        "Location": "",
        "MeasureType": "Advanced",
        "Month": "0",
        "OpponentTeamID": "0",
        "Outcome": "",
        "PORound": "0",
        "PaceAdjust": "N",
        "PerMode": "Totals",
        "Period": "0",
        "PlusMinus": "N",
        "Rank": "N",
        "Season": season,
        "SeasonSegment": "",
        "SeasonType": "Regular Season",
        "ShotClockRange": "",
        "TeamID": team,
        "VsConference": "",
        "VsDivision": ""
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data['resultSets'][0]['rowSet'], columns=data['resultSets'][0]['headers'])
    else:
        print(f"Request failed with status code {response.status_code}")


#### Function to return merged boxscore data

In [4]:
def mergeBoxScoreData(data_std, data_adv):
    return pd.merge(data_std, data_adv.drop(['SEASON_YEAR', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_DATE', \
    'MATCHUP', 'WL', 'MIN', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', \
    'AVAILABLE_FLAG'],axis=1), on='GAME_ID')

#### Run box score collection for all 30 teams for the past 10 seasons

In [10]:
team_ids = [
    "1610612737", "1610612738", "1610612751", "1610612766", "1610612741",
    "1610612739", "1610612742", "1610612743", "1610612765", "1610612744",
    "1610612745", "1610612754", "1610612746", "1610612747", "1610612763",
    "1610612748", "1610612749", "1610612750", "1610612740", "1610612752",
    "1610612760", "1610612753", "1610612755", "1610612756", "1610612757",
    "1610612758", "1610612759", "1610612761", "1610612762", "1610612764"
]

nba_teams = {
    "1610612737": "Atlanta Hawks",
    "1610612738": "Boston Celtics",
    "1610612751": "Brooklyn Nets",
    "1610612766": "Charlotte Hornets",
    "1610612741": "Chicago Bulls",
    "1610612739": "Cleveland Cavaliers",
    "1610612742": "Dallas Mavericks",
    "1610612743": "Denver Nuggets",
    "1610612765": "Detroit Pistons",
    "1610612744": "Golden State Warriors",
    "1610612745": "Houston Rockets",
    "1610612754": "Indiana Pacers",
    "1610612746": "LA Clippers",
    "1610612747": "Los Angeles Lakers",
    "1610612763": "Memphis Grizzlies",
    "1610612748": "Miami Heat",
    "1610612749": "Milwaukee Bucks",
    "1610612750": "Minnesota Timberwolves",
    "1610612740": "New Orleans Pelicans",
    "1610612752": "New York Knicks",
    "1610612760": "Oklahoma City Thunder",
    "1610612753": "Orlando Magic",
    "1610612755": "Philadelphia 76ers",
    "1610612756": "Phoenix Suns",
    "1610612757": "Portland Trail Blazers",
    "1610612758": "Sacramento Kings",
    "1610612759": "San Antonio Spurs",
    "1610612761": "Toronto Raptors",
    "1610612762": "Utah Jazz",
    "1610612764": "Washington Wizards"
}

seasons = ['2023-24', '2022-23', '2021-22', '2020-21', '2019-20', \
    '2018-19', '2017-18', '2016-17', '2015-16', '2014-15']

In [11]:
team_ids = [
    "1610612737", "1610612738"
]
seasons = ['2024-25', '2023-24']

fullData = pd.DataFrame()
for s in range(len(seasons)):
    for t in range(len(team_ids)):
        #first iteration
        if (t==0) and (s==0):
            std = getStdBoxScores(seasons[s], team_ids[t])
            adv = getAdvBoxScores(seasons[s], team_ids[t])
            fullData = mergeBoxScoreData(std, adv)
            print(f"{seasons[s]}, {nba_teams.get(team_ids[t])}")
        else:
            std = getStdBoxScores(seasons[s], team_ids[t])
            adv = getAdvBoxScores(seasons[s], team_ids[t])
            data = mergeBoxScoreData(std, adv)
            fullData = pd.concat([fullData, data])
            print(f"{seasons[s]}, {nba_teams.get(team_ids[t])}")

fullData.to_csv('nba_team_boxscores.csv', index=False)

2024-25, Atlanta Hawks
2024-25, Boston Celtics
2023-24, Atlanta Hawks
2023-24, Boston Celtics
